In [1]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(os.pardir), "src"))

In [2]:
from utils import get_webpage_content, parse_date
from bs4 import BeautifulSoup, Tag
import pandas as pd

## google news

need search query to be like this: news.google.com/search?q=price OR stock OR NSE OR BSE OR share OR Nifty "SBIN" OR "State Bank of India" when%3A1d
- too many unrelated results for SBI PO Jobs, Cut off marks for SBI etc.
- need to use OR operator with financial terms operands to get relevant results
- above query similar to: (price OR stock OR NSE OR BSE OR share OR Nifty) AND ("SBIN" OR "State Bank of India") when:1d

In [48]:
ticker = "SBIN"
ticker_full_name = "State Bank of India"
ticker_alt = "SBI"
financial_terms = ["price", "stock", "NSE", "BSE", "share", "Nifty", "Sensex", "market", ]

In [44]:
query = f'{" OR ".join(financial_terms)} "{ticker}" OR "{ticker_full_name}"'
query

'price OR stock OR NSE OR BSE OR share OR Nifty OR Sensex OR market "SBIN" OR "State Bank of India"'

In [45]:
url = f"https://news.google.com/search?q={query} when:1d".replace(" ", "+").replace('"', '%22')
url

'https://news.google.com/search?q=price+OR+stock+OR+NSE+OR+BSE+OR+share+OR+Nifty+OR+Sensex+OR+market+%22SBIN%22+OR+%22State+Bank+of+India%22+when:1d'

In [46]:
response = get_webpage_content(url, custom_header=False)

In [47]:
soup = BeautifulSoup(response, "html.parser")
article_list = soup.select("article.IFHyqb.DeXSAc")

In [49]:
articles: list[dict[str,str]] = []

for article in article_list:

    headline = article.select_one("a.JtKRv")
    headline = headline.text.strip().replace("\n", " ")
    print(headline)
    if not ticker.lower() in headline.lower() and not ticker_full_name.lower() in headline.lower() and not ticker_alt.lower() in headline.lower():
        continue
    link = article.select_one("a.JtKRv")
    link = link.get("href")
    full_link = f"https://news.google.com{link[1:]}"  # Remove the leading dot from the link
    source_name = article.select_one("div.vr1PYe")
    source_name = source_name.text.strip().replace("\n", " ")
    date_posted_utc = article.select_one("time")
    date_posted_utc = date_posted_utc.get("datetime")
    # date_posted = parse_date(date_posted_utc, relative=False, format="%Y-%m-%dT%H:%M:%S%z")
    articles.append(
        {
            "headline": headline,
            "source_name": source_name,
            "link": full_link,
            "date_posted_utc": date_posted_utc,
        }
    )

NIFTY BANK Index Highlights, 25 April 2025: IDFC First Bank closes below 200-Day Moving Average in today's session
Sensex tanks 1,000 points; Nifty drops below 24,000-level
SBI Life Insurance Shares Jump 10%, Top Nifty Gainer Amid Market Crash: Should You Buy? Check Analysts' Rating
Sensex crashes 1,000 points, Nifty falls below 24,000 amid escalating tensions after Pahalgam attack
SBI Dividend 2025 date: Cash reward in Q4 results FY25 – Check PSU lender’s quarterly earnings announcement schedule
India Stocks Fall, But Head for 2nd Weekly Gains
PNB launches NIRMAAN 2025 to avail of zero processing fees on home, vehicle loans; details here
SBI PO Prelims Result 2025 – sbi.co.in Expected CutOff Marks, Scorecard Download
SBI Personal Loan: Who is eligible and key details you should know
Good News For Borrowers: Bank cuts lending rates


In [50]:
pd.DataFrame(articles)

,headline,source_name,link,date_posted_utc
0,"SBI Life Insurance Shares Jump 10%, Top Nifty ...",Goodreturns,https://news.google.com/read/CBMi3gFBVV95cUxOT...,2025-04-25T06:53:40Z
1,SBI Dividend 2025 date: Cash reward in Q4 resu...,ET Now,https://news.google.com/read/CBMi9wFBVV95cUxNT...,2025-04-25T18:34:50Z
2,SBI PO Prelims Result 2025 – sbi.co.in Expecte...,ofssbihar.in,https://news.google.com/read/CBMiakFVX3lxTFB3R...,2025-04-25T07:11:24Z
3,SBI Personal Loan: Who is eligible and key det...,Moneycontrol,https://news.google.com/read/CBMi1gFBVV95cUxQU...,2025-04-25T05:30:42Z


In [17]:
headline = "SBI SO Salary 2025: Check Assistant Manager Job Profile, Allowances & Monthly Pay in State Bank of India"

In [20]:
not ticker in headline and not ticker_full_name in headline


False

In [12]:
format="%Y-%m-%dT%H:%M:%S%z"

In [14]:
from datetime import datetime
datetime_obj = datetime.strptime('2025-04-20T06:34:35Z', format)

In [15]:
datetime_obj.tzinfo

datetime.timezone.utc

In [102]:
format.__contains__("%Y")

True

In [70]:

print(f"Headline: {headline}")
print(f"Full Link: {full_link}")
print(f"Source Name: {source_name}")
print(f"Date Posted (UTC): {date_posted_utc}")

Headline: India’s electronics sector eyes $500 bn mark by 2030; Kaynes Technology, Dixon Technologies top buys
Full Link: https://news.google.com/read/CBMi_wFBVV95cUxNQTVuWFE3c1VJdGhkampRX2prQzZlYmFuRnh1X01PMi1BalRwTFRJMXlJSFdPWFZoYWhVMnVkVXE1dnZzTm9rX0NUc1l2YUx2VEllekdXaHV0Yzc1d19mZW5KMFJ1bVVJWVMzTUdWV2s5bFZlejZQUXI0dWkyMDNPTXV5b2ZKSGVuWEZQcmhERFI4bVVmZW5YaXpHckwtWk9PSmJldG8tVVN6dF9PeUluVmdlNGJvS0pfZGlzRDJDbkJBN0NJZ1pKVmZCaTRiMXdEVk9DUl92RkFycllKNU52ZlRfUGxIVXZ5Z1dEUGdEbHVPaV9IZjlxSzlYUThQUFXSAYQCQVVfeXFMTTVTZnROMWxwYmtnUGtybldiRkNrOC1nSlNLVGtzQUlrM05TSFFjdUlxYXVaRm1rOFduWjRfajVaQVVGT2Y1Y3VMQmNJQ1hTdmNvNU14VjZNNWN0U0dHMi1hYmVuUzdMd1k2WGdDVGdXZ1NERGZ3VFpWdUJWZDdMb1J1WXBBT01UZldVTlJvTlBpcjRKS2QtcXY3UE9xZTBOdmVxTThSdUlhQlQyNzdCTDNIMFU5Vl9xNm5OOHJ2OUpMMWp5MERRSlVxZmpvcDdfbVRZX0c4MmJZZmstdXhVSVBzdGRQZF9DVFlKVmdCMGVBbC1RY3BoX2hiU2NVR1IxTGxGZmw?hl=en-IN&gl=IN&ceid=IN%3Aen
Source Name: The Economic Times
Date Posted (UTC): 2025-04-20T06:34:35Z


In [49]:
full_content = get_webpage_content(full_link, custom_header=False)

In [52]:
import httpx
from bs4 import BeautifulSoup
import re
import urllib.parse

google_news_url = full_link

# Use a common browser User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

final_url = None
try:
    # Fetch the initial page content WITHOUT following redirects
    response = httpx.get(google_news_url, headers=headers, follow_redirects=False, timeout=10)
    response.raise_for_status()

    # Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Look for the target link (this selector might need adjustment if Google changes its structure)
    # Often the link is in an 'a' tag near the top
    link_element = soup.find('a', {'rel': 'nofollow'}) # A common pattern, but might vary
    if link_element and link_element.get('href'):
        final_url = link_element['href']
    else:
        # Fallback: Sometimes the URL is embedded in JavaScript
        script_tags = soup.find_all('script')
        for script in script_tags:
            if script.string:
                # Look for patterns like window.location.replace or similar
                match = re.search(r'window\.location\.replace\([\'"](http[s]?://.*?)[\'"]\)', script.string)
                if match:
                    final_url = urllib.parse.unquote(match.group(1)) # Decode URL if needed
                    break # Found it

    if final_url:
        print(f"Extracted Final URL: {final_url}")
        # Now you can use httpx.get(final_url, ...) to fetch the actual article content
        # article_response = httpx.get(final_url, headers=headers, follow_redirects=True)
        # print(article_response.url) # Verify the final URL after potential redirects from the target site
    else:
        print("Could not automatically extract the final URL from the Google News page.")
        print("The page structure might have changed, or it requires JavaScript execution.")
        # You might need to inspect response.text manually or consider tools like Selenium/Playwright

except httpx.RequestError as exc:
    print(f"An error occurred while requesting {exc.request.url!r}: {exc}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# ...existing code...
# Replace the old print statements with the extracted URL if found
# print(f"Headline: {headline}") # Keep if needed
# print(f"Full Link: {full_link}") # Replace this with the extracted final_url

if final_url:
    print(f"\nAttempting to fetch content from: {final_url}")
    try:
        # Fetch content from the extracted final URL
        final_content_response = httpx.get(final_url, headers=headers, follow_redirects=True, timeout=20)
        final_content_response.raise_for_status()
        print(f"Successfully fetched content from: {final_content_response.url}") # Show the URL after final redirects
        # full_content = final_content_response.text # Process the content
    except httpx.RequestError as exc:
        print(f"Failed to fetch final URL content: {exc}")
    except Exception as e:
        print(f"An error occurred fetching final content: {e}")

Could not automatically extract the final URL from the Google News page.
The page structure might have changed, or it requires JavaScript execution.


## stocktwits

https://firestream-portal.stocktwits.com/documentation
https://api.stocktwits.com/developers/docs/api/stream/get_streams

In [18]:
symbol = "DIXON.NSE"
api_url = f"https://api.stocktwits.com/api/2/streams/symbol/{symbol}.json"

In [19]:
api_url

'https://api.stocktwits.com/api/2/streams/symbol/DIXON.NSE.json'

In [23]:
import json
import httpx


response = httpx.get(api_url)
data = response.json()

In [24]:
data

{'symbol': {'id': 20412,
  'symbol': 'DIXON.NSE',
  'symbol_mic': 'DIXON.XNSE',
  'exchange': 'NSE',
  'region': 'IN',
  'title': 'Dixon Technologies (India) Ltd',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 5409,
  'has_pricing': True,
  'instrument_class': 'Stock',
  'live_event': False,
  'cusip': 'Y2R8Y5112',
  'isin': 'INE935N01020',
  'should_seo_index': True},
 'cursor': {'more': True, 'since': 612386598, 'max': 610374664},
 'regions': ['IN'],
 'filters': [],
 'messages': [{'id': 612386598,
   'body': '@StocktwitsIndia  \n$DIXON.NSE',
   'created_at': '2025-04-23T02:01:23Z',
   'discussion': False,
   'user': {'id': 9067762,
    'username': 'Rookie_Trader99',
    'name': 'Sunny Verma',
    'avatar_url': 'https://avatars.stocktwits.com/production/9067762/thumb-1715752795.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/9067762/thumb-1715752795.png',
    'join_date': '2024-05-12',
    'official': False,
    'identity': 'User',
    'classifi

In [1]:
import requests

url = "https://stocktwits.p.rapidapi.com/streams/symbol/AAPL.json"

querystring = {"limit":"20"}

headers = {"x-rapidapi-host": "stocktwits.p.rapidapi.com"}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'Invalid API key. Go to https://docs.rapidapi.com/docs/keys for more info.'}
